In [1]:
# imports

import numpy as np
import math

# Define new bounding boxes for cases where image basis changes

Note: point rotations, filtering preserves the original coordinates of the target object, no need to redefine bounding boxes after augmentation

image label(s): `<class_id> <x_center> <y_center> <width> <height>`
-  All values of bounding box related to image are normalized by image width and height (values between 0 and 1).


### parse label items

In [ ]:
def parse_label_string(label_string):
    """
    Parses a YOLO label string into components.
    
    Parameters:
    - label_string: str, format '<class_id> <x_center> <y_center> <width> <height>'
    
    Returns:
    - (class_id, x_center, y_center, width, height): tuple with class_id as str, others as floats
    """
    parts = label_string.strip().split()
    class_id = parts[0]
    x_center = float(parts[1])
    y_center = float(parts[2])
    width = float(parts[3])
    height = float(parts[4])
    return class_id, x_center, y_center, width, height

def format_label(class_id, x_center, y_center, width, height):
    """
    Formats label components back into YOLO string format.
    """
    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

### bounding boxes for reflections

In [ ]:
def reflect_over_x_bbox(label_string):
    """
    Reflects bounding box over x-axis (vertical flip).
    """
    class_id, x_center, y_center, width, height = parse_label_string(label_string)
    new_y_center = 1.0 - y_center
    return format_label(class_id, x_center, new_y_center, width, height)

In [ ]:
def reflect_over_y_bbox(label_string):
    """
    Reflects bounding box over y-axis (horizontal flip).
    """
    class_id, x_center, y_center, width, height = parse_label_string(label_string)
    new_x_center = 1.0 - x_center
    return format_label(class_id, new_x_center, y_center, width, height)

### bounding boxes for rotations

In [ ]:
def rotate_bbox(label_string, image_shape, angle_degrees):
    """
    Rotates a YOLO-format bounding box around the image center and returns a new axis-aligned bbox.
    
    Parameters:
    - label_string: str, YOLO format '<class_id> <x_center> <y_center> <width> <height>'
    - image_shape: tuple, (height, width) of the image
    - angle_degrees: float, angle to rotate (clockwise)
    
    Returns:
    - new_label_string: str, new YOLO-format bounding box
    """
    class_id, xc, yc, w, h = parse_label_string(label_string)
    img_h, img_w = image_shape

    # Convert to absolute coordinates
    xc *= img_w
    yc *= img_h
    w *= img_w
    h *= img_h

    # Get 4 corners of the bounding box
    x1 = xc - w / 2
    y1 = yc - h / 2
    x2 = xc + w / 2
    y2 = yc + h / 2
    corners = np.array([
        [x1, y1],
        [x1, y2],
        [x2, y1],
        [x2, y2]
    ])

    # Rotation matrix
    angle_rad = math.radians(angle_degrees)
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)

    # Image center
    cx, cy = img_w / 2, img_h / 2

    # Rotate each corner
    rotated = []
    for x, y in corners:
        # shift to origin
        x -= cx
        y -= cy
        # rotate
        xr = x * cos_a - y * sin_a
        yr = x * sin_a + y * cos_a
        # shift back
        xr += cx
        yr += cy
        rotated.append([xr, yr])

    rotated = np.array(rotated)

    # Get new axis-aligned bounding box
    x_min, y_min = rotated.min(axis=0)
    x_max, y_max = rotated.max(axis=0)

    # Re-normalize
    new_xc = (x_min + x_max) / 2 / img_w
    new_yc = (y_min + y_max) / 2 / img_h
    new_w = (x_max - x_min) / img_w
    new_h = (y_max - y_min) / img_h

    return format_label(class_id, new_xc, new_yc, new_w, new_h)

### bounding boxes for shears

In [ ]:
def shear_bbox(label_string, image_shape, shear_factor_x=0.0, shear_factor_y=0.0):
    """
    Apply shear to a YOLO bounding box and return the updated YOLO-format string.

    Parameters:
    - label_string: str, YOLO format '<class_id> <x_center> <y_center> <width> <height>'
    - image_shape: tuple, (height, width)
    - shear_factor_x: horizontal shear factor
    - shear_factor_y: vertical shear factor

    Returns:
    - new_label_string: str, updated bounding box in YOLO format
    """
    class_id, xc, yc, w, h = parse_label_string(label_string)
    img_h, img_w = image_shape

    # Convert normalized to absolute coordinates
    xc *= img_w
    yc *= img_h
    w *= img_w
    h *= img_h

    # Get original corners
    x1 = xc - w / 2
    y1 = yc - h / 2
    x2 = xc + w / 2
    y2 = yc + h / 2
    corners = np.array([
        [x1, y1],
        [x1, y2],
        [x2, y1],
        [x2, y2]
    ])

    # Apply shear transformation
    sheared = []
    for x, y in corners:
        new_x = x + shear_factor_x * y
        new_y = y + shear_factor_y * x
        sheared.append([new_x, new_y])

    sheared = np.array(sheared)

    # Compute axis-aligned bounding box
    x_min, y_min = sheared.min(axis=0)
    x_max, y_max = sheared.max(axis=0)

    # Convert back to normalized
    new_xc = (x_min + x_max) / 2 / img_w
    new_yc = (y_min + y_max) / 2 / img_h
    new_w = (x_max - x_min) / img_w
    new_h = (y_max - y_min) / img_h

    return format_label(class_id, new_xc, new_yc, new_w, new_h)
